In [1]:
import gradio as gr
from main import Chatbot as AI
ai = AI()

def chatbot_response(message, history):
    """
    Hàm xử lý tin nhắn chatbot.
    message: Tin nhắn từ người dùng.
    history: Lịch sử hội thoại (list chứa các tuple (user, bot)).
    """
    his = ""
    for m, r in history[::-1]:
        his = "SIUBOT:" + "\n" + r  + "\n" +  his
        his = "USER: " + "\n" + m + "\n" + his + "\n"
        if len(his)>8000:
            break
    inp = f"""

    {his}
    # Tin nhắn hiện tại của phụ huynh hoặc học sinh:
    {message}
    """
    response, debug_info = ai.respond(inp)
    history = history + [(message, response)] 
    return "", history, debug_info, history  # Trả về lịch sử mới để cập nhật state

# Tạo giao diện Gradio
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # with gr.Row():
    
    chatbot = gr.Chatbot(height=700, autoscroll=True)
        
    
    msg = gr.Textbox(label="Nhập tin nhắn")
    submit_btn = gr.Button("Gửi")
    debug_output = gr.Textbox(label="Thông tin xử lý")
    state = gr.State([])  # Lưu lịch sử hội thoại
    
    def on_submit(message, history):
        return chatbot_response(message, history)
    
    submit_btn.click(on_submit, inputs=[msg, state], outputs=[msg, chatbot, debug_output, state])
    msg.submit(on_submit, inputs=[msg, state], outputs=[msg, chatbot, debug_output, state])

demo.launch(share=True)


/workspace/nhihtc/anaconda/miniconda3/envs/shine_nckh1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_761963/3076852484.py:31: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=700, autoscroll=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3275aa7dbd7888a467.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import pandas as pd
import os
import time
import uuid
from main import Chatbot as AI

ai = AI()

# Tạo thư mục lưu lịch sử nếu chưa có
if not os.path.exists("chat_logs"):
    os.makedirs("chat_logs")

def generate_session_id():
    """Tạo ID phiên làm việc dựa trên timestamp + UUID"""
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    unique_id = uuid.uuid4().hex[:6]  # Lấy 6 ký tự từ UUID ngẫu nhiên
    return f"{timestamp}_{unique_id}"

def save_to_csv(user_message, bot_response, session_id):
    """Lưu hội thoại vào file CSV riêng theo session ID"""
    file_path = f"chat_logs/chat_{session_id}.csv"
    
    new_data = pd.DataFrame([[user_message, bot_response]], columns=["User", "Bot"])

    # Ghi dữ liệu vào file CSV, không ghi đè dữ liệu cũ
    new_data.to_csv(file_path, mode="a", index=False, header=not os.path.exists(file_path), encoding="utf-8")

def chatbot_response(message, history, session_id):
    """
    Xử lý tin nhắn chatbot và lưu vào file hội thoại riêng (CSV).
    """
    his = ""
    for m, r in history[::-1]:
        his = "SIUBOT:" + "\n" + r  + "\n" +  his
        his = "USER: " + "\n" + m + "\n" + his + "\n"
        if len(his) > 8000:
            break

    inp = f"""
# Hội thoại 
{his}

# Tin nhắn hiện tại :
USER: {message}
"""
    response, debug_info = ai.respond(inp)
    
    # Lưu vào file CSV riêng theo session ID
    save_to_csv(message, response, session_id)

    history = history + [(message, response)]
    return "", history, debug_info, history  # Trả về lịch sử mới để cập nhật state

def initialize_session():
    """Chỉ tạo session ID khi mở giao diện, không có tin nhắn chào"""
    return "", generate_session_id()  # Trả về session ID

# Tạo giao diện Gradio
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    chatbot = gr.Chatbot(height=700, autoscroll=True)
    msg = gr.Textbox(label="Nhập tin nhắn")
    submit_btn = gr.Button("Gửi")
    debug_output = gr.Textbox(label="Thông tin xử lý")
    state = gr.State([])  # Lưu lịch sử hội thoại
    session_id = gr.State("")  # Khởi tạo session ID rỗng

    def on_submit(message, history, session_id):
        return chatbot_response(message, history, session_id)

    # Khi mở giao diện, bot nhắn trước và tạo session ID
    demo.load(initialize_session, inputs=[], outputs=[debug_output, session_id])

    submit_btn.click(on_submit, inputs=[msg, state, session_id], outputs=[msg, chatbot, debug_output, state])
    msg.submit(on_submit, inputs=[msg, state, session_id], outputs=[msg, chatbot, debug_output, state])

demo.launch(share=True)


/workspace/nhihtc/anaconda/miniconda3/envs/shine_nckh1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_969199/3985090501.py:61: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=700, autoscroll=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d7cc28220129bbe0e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**user**

# Hội thoại 


# Tin nhắn hiện tại :
USER: xin chào

----------------------------------------
**Chatbot**
start
----------------------------------------
**Function**
├── Các câu hỏi về chatbot và câu hỏi ngoài lề.md
├── Học bổng
├── Psychology
├── THÔNG TIN CHUNG
└── Thông tin tuyển sinh
    ├── Chính sách ưu tiên xét tuyển thẳng, ưu tiên xét tuyển.md
    ├── Các ngành xét tuyển.md
    ├── Các phương thức xét tuyển
    ├── Học phí.md
    ├── Phạm vi tuyển sinh.md
    ├── thông tin đăng ký, nộp hồ sơ và xét tuyển vào trường SIU
    │   ├── Lệ phí xét tuyển 2024.md
    │   ├── Quy trình nộp hồ sơ xét tuyết với thí sinh dự thi THPT 2024.md
    │   ├── Quy trình nộp hồ sơ xét tuyết với thí sinh tốt nghiệp THPT dựa trên kết quả trung học 2024.md
    │   ├── Quy trình nộp hồ sơ xét tuyết với thí sinh đã tốt nghiệp THPT ở ngước ngoài 2024.md
    │   ├── Quy định, điều kiện về hồ sơ đăng ký xét tuyển.md
    │   └── Thời gian xét tuyển.md
    └── Điểm trúng tuyển
---------------------